In [2]:
# CONECTAR CON DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import sys
path ='/content/drive/MyDrive/cod/LEA3_Marketing'
os.chdir(path) ## volver la carpeta de repositorio directorio de trabajo
sys.path.append(path) ## agregarla al path, para leer archivos propios como paquetes

In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
#### conectar_base_de_Datos
#!pip install ipywidgets

# CREAR CONEXIÓN CON LA BASE DE DATOS db_movies
con = sql.connect('data/db_movies')

# CREAR EL CURSOR
cur = con.cursor() ## se crea el cursor, que es el otro tipo de conexión para ejecutar las consultas

In [2]:
# VERIFICAR LOS NOMBRES DE TODAS LAS TABLAS QUE HAY EN LA BASE DE DATOS
cur.execute(""" select name from sqlite_master where type= 'table'  """)
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_selectos',),
 ('Pelis_selectas',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',)]

In [3]:
movies_final=pd.read_sql_query('SELECT * FROM movies_final', con)

In [25]:
movies_final

,movie_id,movie_title,movie_genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,5,Father of the Bride Part II (1995),Comedy
4,6,Heat (1995),Action|Crime|Thriller
...,...,...,...
2116,174055,Dunkirk (2017),Action|Drama|Thriller|War
2117,176371,Blade Runner 2049 (2017),Sci-Fi
2118,177765,Coco (2017),Adventure|Animation|Children
2119,179819,Star Wars: The Last Jedi (2017),Action|Adventure|Fantasy|Sci-Fi


In [12]:
## consulta de peliculas que estan en la tabla movies, que no tienen calificacion alguna en la tabla ratings
df_delete = pd.read_sql(
    """SELECT * FROM movies_final
    LEFT JOIN ratings ON movies_final.movie_id = ratings.movieId
    WHERE ratings.rating IS NULL""", con)

In [43]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from mlxtend.preprocessing import TransactionEncoder

# Paso 0: df_delete ya obtenido previamente con SQL

# Función 1: separar géneros y convertir a binario con TransactionEncoder
def split_and_encode_genres(df):
    genres = df['movie_genres'].str.split('|')
    te = TransactionEncoder()
    genres_bin = te.fit_transform(genres)
    genres_df = pd.DataFrame(genres_bin, columns=te.columns_)

    # Eliminar "(no genres listed)" si existe
    if '(no genres listed)' in genres_df.columns:
        valid_rows = ~genres_df['(no genres listed)'] # La virgulilla me convierte lo TRUE en FALSE y viceversa
        df = df.loc[valid_rows].reset_index(drop=True) # Filtro por las columnas que si tienen genero
        genres_df = genres_df.loc[valid_rows].drop(columns='(no genres listed)').reset_index(drop=True)

    # Eliminar columna original 'genres' y unir los géneros codificados
    df = df.drop(columns='movie_genres').reset_index(drop=True) # Elimina la columna original 'genres' del df
    return pd.concat([df, genres_df], axis=1)

# Función 2: eliminar registros que estén en df_delete
def remove_unrated_movies(df):
    return df[~df['movie_id'].isin(df_delete['movie_id'])].reset_index(drop=True)

# Función 3: extraer título y año
def extract_title_and_year(df):
    year = df['movie_title'].str.extract(r'\((\d{4})\)$')
    year.columns = ['movie_year']
    title = df['movie_title'].str.replace(r'\s*\(\d{4}\)$', '', regex=True)
    title.name = 'movie_title'
    df = df.drop(columns='movie_title')
    df = pd.concat([df.reset_index(drop=True), title.reset_index(drop=True), year.reset_index(drop=True)], axis=1)
    return df

# Función 4: eliminar registros con year == NaN
def remove_nan_years(df):
    return df[df['movie_year'].notna()].reset_index(drop=True)

# Función 5: Reordenar columnas para que 'title' y 'year' estén después de 'movieId'
def reorder_columns(df):
    cols = list(df.columns)
    if 'movie_id' in cols and 'movie_title' in cols and 'movie_year' in cols:
        cols.remove('movie_title')
        cols.remove('movie_year')
        insert_pos = cols.index('movie_id') + 1
        cols[insert_pos:insert_pos] = ['movie_title', 'movie_year']
    return df[cols]

# Construcción del pipeline
pipeline = Pipeline(steps=[
    ('genres_transform', FunctionTransformer(split_and_encode_genres, validate=False)),
    ('remove_unrated', FunctionTransformer(remove_unrated_movies, validate=False)),
    ('extract_title_year', FunctionTransformer(extract_title_and_year, validate=False)),
    ('remove_nan_years', FunctionTransformer(remove_nan_years, validate=False)),
    ('reorder_columns', FunctionTransformer(reorder_columns, validate=False))  # Nuevo paso
])

# Aplicar el pipeline
db_movies_final = pipeline.fit_transform(movies_final)

In [44]:
db_movies_final

,movie_id,movie_title,movie_year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,1995,False,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2,Jumanji,1995,False,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,3,Grumpier Old Men,1995,False,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
3,5,Father of the Bride Part II,1995,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,6,Heat,1995,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2116,174055,Dunkirk,2017,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
2117,176371,Blade Runner 2049,2017,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2118,177765,Coco,2017,False,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2119,179819,Star Wars: The Last Jedi,2017,True,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [45]:
sc = MinMaxScaler()
db_movies_final[["year_sc"]] = sc.fit_transform(db_movies_final[['movie_year']])

In [49]:
db_movies_final

,movie_id,movie_title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year_sc
0,1,Toy Story,False,True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,0.760417
1,2,Jumanji,False,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0.760417
2,3,Grumpier Old Men,False,False,False,False,True,False,False,False,...,False,False,False,False,True,False,False,False,False,0.760417
3,5,Father of the Bride Part II,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,0.760417
4,6,Heat,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,0.760417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2116,174055,Dunkirk,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,True,True,False,0.989583
2117,176371,Blade Runner 2049,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,0.989583
2118,177765,Coco,False,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0.989583
2119,179819,Star Wars: The Last Jedi,True,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,0.989583


In [48]:
db_movies_final=db_movies_final.drop(columns=['movie_year'])

In [ ]:
pelicula ='Toy Story'
ind_peli=books[books['book_title']==libro].index.values.astype(int)[0] ### indice del libro en dataframe escalado y dummificado
row_sel_book =books_dum2.iloc[ind_libro,:] ### seleccionar fila de caracteristicas de libro seleccionados
similar_books=books_dum2.corrwith(row_sel_book,axis=1) ##calcular correlación de catalogo vs libro seleccionado
similar_books=similar_books.sort_values(ascending=False) ### ordener libros de mayor a menor correlación
top_similar_books=similar_books.to_frame(name="correlación").iloc[0:11,] ### el 11 es número de libros recomendados
top_similar_books['book_title']=books["book_title"] ### agregaro los nombres (como tiene mismo indice no se debe cruzar)



In [31]:
from sklearn import neighbors
from ipywidgets import interact
import pandas as pd

# 1. Seleccionamos solo las columnas de género (todas numéricas)
X = db_movies_final.drop(columns=['movie_id', 'movie_title', 'movie_year'])

# 2. Creamos y entrenamos el modelo con distancia del coseno
model = neighbors.NearestNeighbors(n_neighbors=20, metric='cosine')
model.fit(X)

# 3. Obtenemos los vecinos más cercanos de cada película
dist, idlist = model.kneighbors(X)

# 4. Guardamos como DataFrame para inspección opcional
distancias = pd.DataFrame(dist)
id_list = pd.DataFrame(idlist)

In [32]:
def MovieRecommender(movie_name = list(db_movies_final['movie_title'].value_counts().index)):
    movie_list_name = []

    # Índice de la película original
    movie_id = db_movies_final[db_movies_final['movie_title'] == movie_name].index
    if len(movie_id) == 0:
        return ["Película no encontrada"]
    movie_id = movie_id[0]

    # Vector binario de la película original
    base_vector = db_movies_final.loc[movie_id].drop(['movie_id', 'movie_title', 'movie_year'])

    for newid in idlist[movie_id]:
        if newid == movie_id:
            continue  # Saltar si es la misma película
        candidate_vector = db_movies_final.loc[newid].drop(['movie_id', 'movie_title', 'movie_year'])

        # Contar coincidencias de género
        matches = (base_vector & candidate_vector).sum()

        if matches >= 2:
            movie_list_name.append(db_movies_final.loc[newid, 'movie_title'])

    return movie_list_name

# Ejecutar
interact(MovieRecommender)

interactive(children=(Dropdown(description='movie_name', options=('King Kong', 'Jungle Book, The', 'Planet of …

<function __main__.MovieRecommender(movie_name=['King Kong', 'Jungle Book, The', 'Planet of the Apes', 'Metropolis', 'Dawn of the Dead', 'Stepford Wives, The', 'Father of the Bride', 'Manchurian Candidate, The', 'Nutty Professor, The', 'Parent Trap, The', 'Crash', 'Alice in Wonderland', 'Hairspray', 'Fly, The', 'Miracle on 34th Street', 'Sabrina', 'Cape Fear', 'Godzilla', 'Avengers, The', 'Spider-Man', 'Secretary', 'Das Experiment (Experiment, The)', 'About a Boy', 'Igby Goes Down', 'Star Wars: Episode II - Attack of the Clones', 'Insomnia', 'Transporter, The', 'Hot Shots!', 'One Hour Photo', 'xXx', 'Good Girl, The', 'Signs', 'Sum of All Fears, The', 'Austin Powers in Goldmember', 'Road to Perdition', 'Reign of Fire', 'Men in Black II (a.k.a. MIIB) (a.k.a. MIB 2)', 'Mr. Deeds', 'Minority Report', 'Lilo & Stitch', 'Scooby-Doo', 'Bourne Identity, The', 'Spirited Away (Sen to Chihiro no kamikakushi)', 'Hours, The', 'Chicago', 'Catch Me If You Can', 'My Neighbor Totoro (Tonari no Totoro)', 'Two Weeks Notice', 'Gangs of New York', '25th Hour', 'Lord of the Rings: The Two Towers, The', 'About Schmidt', 'Star Trek: Nemesis', 'Maid in Manhattan', 'Equilibrium', 'Adaptation', 'Solaris', 'Bowling for Columbine', 'Die Another Day', 'Harry Potter and the Chamber of Secrets', 'Far from Heaven', '8 Mile', 'Frida', 'Jackass: The Movie', 'Professional, The (Le professionnel)', 'Saturday Night Fever', 'Grave of the Fireflies (Hotaru no haka)', 'Ring, The', 'Punch-Drunk Love', 'Strange Brew', 'Red Dragon', 'Sweet Home Alabama', 'Talk to Her (Hable con Ella)', 'Lord of the Rings: The Fellowship of the Ring, The', 'Kate & Leopold', "Bill & Ted's Bogus Journey", 'Royal Tenenbaums, The', 'Vanilla Sky', 'Not Another Teen Movie', "Amelie (Fabuleux destin d'Amélie Poulain, Le)", "No Man's Land", "Ocean's Eleven", 'Behind Enemy Lines', "Ocean's Eleven (a.k.a. Ocean's 11)", 'Spy Game', "Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone)", 'Shallow Hal', 'One, The', 'Rashomon (Rashômon)', "Man Who Wasn't There, The", 'Donnie Darko', 'Thirteen Ghosts (a.k.a. Thir13en Ghosts)', 'K-PAX', 'Waking Life', 'From Hell', 'Dirty Harry', 'Mulholland Drive', 'Serendipity', 'Zoolander', 'Training Day', 'Jay and Silent Bob Strike Back', 'Rat Race', 'Others, The', 'American Pie 2', 'Monsters, Inc.', 'The Scorpion King', 'Three Men and a Baby', 'My Big Fat Greek Wedding', 'Frailty', 'Pianist, The', "National Lampoon's Van Wilder", 'Rookie, The', 'Panic Room', 'Death to Smoochy', 'Blade II', 'And Your Mother Too (Y tu mamá también)', 'Resident Evil', 'Ice Age', 'Time Machine, The', 'We Were Soldiers', 'Beautiful Mind, A', 'Black Hawk Down', 'Gosford Park', 'I Am Sam', "Monster's Ball", 'Orange County', 'Brotherhood of the Wolf (Pacte des loups, Le)', '48 Hrs.', '40 Days and 40 Nights', 'The Count of Monte Cristo', 'Mothman Prophecies, The', 'Walk to Remember, A', 'Sandlot, The', 'John Q', 'Super Troopers', 'Queen of the Damned', 'Monsoon Wedding', 'M*A*S*H (a.k.a. MASH)', 'Starsky & Hutch', 'Hidalgo', 'Good bye, Lenin!', 'Passion of the Christ, The', 'EuroTrip', '50 First Dates', 'Dreamers, The', 'The Butterfly Effect', 'Chitty Chitty Bang Bang', 'Along Came Polly', 'Paycheck', 'Cold Mountain', 'Monster', 'Fog of War: Eleven Lessons from the Life of Robert S. McNamara, The', 'Lord of the Rings: The Return of the King, The', 'How to Lose a Guy in 10 Days', 'Matrix Revolutions, The', 'Elf', 'Love Actually', 'Master and Commander: The Far Side of the World', '21 Grams', 'Bad Santa', 'WarGames', 'Girl with a Pearl Earring', 'Kindergarten Cop', 'Battle Royale (Batoru rowaiaru)', 'Witches of Eastwick, The', 'Hero (Ying xiong)', 'Nausicaä of the Valley of the Wind (Kaze no tani no Naushika)', 'Last Samurai, The', 'Big Fish', "Something's Gotta Give", 'Hannah and Her Sisters', 'Anchorman: The Legend of Ron Burgundy', 'King Arthur', 'Before Sunset', 'Spider-Man 2', 'Roxanne', 'Fahrenheit 9/11', 'Notebook, The', 'Terminal, The', 'Dodgeball: A T

In [41]:
db_movies_final = db_movies_final.drop(columns=['movie_id', 'movie_year'])


# Seleccionamos una película de ejemplo
pelicula = 'Toy Story'

# Obtener índice de la película en el DataFrame original (que contiene títulos y dummies con mismos índices)
ind_pelicula = db_movies_final[db_movies_final['movie_title'] == pelicula].index.values.astype(int)[0]

# Seleccionar la fila dummificada correspondiente
row_sel_movie = db_movies_final.iloc[ind_pelicula, :]

# Calcular la correlación con todas las demás películas
similar_movies = db_movies_final.corrwith(row_sel_movie, axis=1)

# Ordenar las películas por similitud descendente
similar_movies = similar_movies.sort_values(ascending=False)

# Tomar el top 10 (excluyendo la misma película)
top_similar_movies = similar_movies.to_frame(name="correlación").iloc[1:11, :]  # desde el 1 para excluir la misma

# Agregar los títulos de películas
top_similar_movies['movie_title'] = db_movies_final.loc[top_similar_movies.index, 'movie_title']
top_similar_movies['movie_year'] = db_movies_final.loc[top_similar_movies.index, 'movie_year']

top_similar_movies

ValueError: could not convert string to float: 'Toy Story'